## Using Earsketch for placing audio samples on a temporal grid

**Given:** a list of up to 10 samples and a trigger map mapping an onset to one of the samples. Every onset stops the playback of a previous sample that might still be playing (several layers can be created that way if you want samples to overlap)

**Goal:** an audio file that contains the samples at the desired positions.

This process can be used to plug together your entire composition, or for producing new samples that can then be used in the next iteration.

### 1. Upload samples to Earsketch

* Head to https://earsketch.gatech.edu/earsketch2/
* In order to upload, you need to create an account.
* On the left side, open the "Sounds" tab and click "Add sound". You should see the upload mask:

<img src="earsketch_upload.png">

* under "Constant Value" add a meaningful name. Note that it will be capitalized and your username will be prepended.
* Scroll down in the sound collection until you see the category with your username. Here you find the names of all your uploaded samples which you can use as variables in the Python code on the right.

### Combine samples and triggermap

<img src="earsketch_playback.png">

* In the code field, the first three lines and the last one always need to be there. We set the tempo to 60 bpm, so that every beat has the duration of 1 second.
* Then we define the list with the `samples` that will be triggered and note the list index of each.
* Now we define a triggermap `onsets`:
  * every position in the string corresponds to 1/4 of a beat, so 0.25 seconds is the lowest resolution (in this tempo)
  * an integer 0-9 causes the sample at this index to be played back
  * every `+` causes the playback to continue, every `-` stands for a rest
* then we combine the `samples` list with the `onsets`, using the function `makeBeat()`. The arguments are:
  * a single sample or list of samples
  * the track (layer) in which the samples are to be played
  * the measure in which to begin the playback
  * the triggerlist
* once defined, we can hit the green "Run" button and see the outcome on top in the Digital Audio Workstation (DAW)
* in the example, you see that
  * the triggermap `onsets` contains 1 for the second sample, `JEYES_GOAT` and 0 for the first sample, `JEYES_HARM`
  * it is put into the first track at measure 2
  * the second call to the `makeBeat` function only receives one sample and therefore only uses 0 in the triggermap, which also contains rests, i.e, for `0-`, only the first 0.25 seconds of the sample are being played.
  
Here are a few lines of code to give you an idea how you can translate your durations expressed in seconds to such a triggermap:

In [16]:
def duration2earsketch(dur, tempo=60):
    """For a duration in seconds and a tempo, return the number of symbols to put into the trigger map."""
    resolution = 4 * round(tempo) / 60
    return round(float(dur) * resolution)

def earsketch_event(sample_no, positions, sounding_positions=None):
    """Pass the index of the sample to be played, the number you got from duration2earsketch,
    and how many of the positions you want the sample to keep sounding (the rest is filled up with rests)."""
    if sample_no is None:
        return '-' * positions
    positions -= 1
    sounding_positions = positions if sounding_positions is None else sounding_positions - 1
    rest_positions = positions - sounding_positions
    return f"{sample_no}{'+' * sounding_positions}{'-' * rest_positions}"

def durations2triggermap(succession_of_samples, onsets, tempo=60):
    if isinstance(succession_of_samples, int):
        succession_of_samples = [succession_of_samples]
    nxt_sample = (s for s in succession_of_samples)
    result = ''
    for ons in onsets:
        positions = duration2earsketch(ons)
        if isinstance(ons, str):
            result += earsketch_event(None, positions)
        else:
            try:
                sample_no = next(nxt_sample)
            except:
                break
            result += earsketch_event(sample_no, positions)
    return result
            
sample_succession = [0, 1, 0]
durations_in_seconds = [0, '1', 1.75, 6.9]    
durations2triggermap(sample_succession, durations_in_seconds)

'0----1++++++0+++++++++++++++++++++++++++'